In [120]:
import json
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt

In [121]:
def check_avg(df_, season, team, stats = ['AST']):
    avg_stats = ['AVG_'+stat for stat in stats]
    df = df_.loc[(df_['SEASON_ID'] == season)& (df_['TEAM_ABBREVIATION'] == team)].sort_values('GAME_DATE')
    avg = df[stats].expanding().mean().shift(1)
    diff = df[avg_stats].values - avg[stats].values
    return(np.max(diff[1:]), np.min(diff[1:]))

In [309]:
STATS = ['PTS', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'DREB', 'OREB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
AVG_STATS = ['AVG_'+stat for stat in STATS]
OPP_AVG_STATS = ['OPP_'+stat for stat in AVG_STATS]

In [315]:
df = pd.read_csv('dat/1983-2017.csv', index_col=0).sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE'])
df = df.set_index('GAME_ID')
df['FG3A'] = df['FG3A'].fillna(0)
df.loc[df['FG3A']==0, ['FG3M', 'FG3_PCT']] = 0
df['FG2M'] = df['FGM'] - df['FG3M']
df['FG2A'] = df['FGA'] - df['FG3A']
df['FG2_PCT'] = df['FG2M'] / df['FG2A']
df['PLUS_MINUS'] = df['PTS'] - df['PTS_A']
df = df.reset_index()

In [316]:
for season in df.loc[df[STATS].isna().any(axis=1)]['SEASON_ID'].values:
    df = df.loc[df['SEASON_ID']!=season]

In [317]:
df[STATS].isna().any(axis=0)

PTS        False
FG2M       False
FG2A       False
FG2_PCT    False
FG3M       False
FG3A       False
FG3_PCT    False
FTM        False
FTA        False
FT_PCT     False
DREB       False
OREB       False
AST        False
STL        False
BLK        False
TOV        False
PF         False
dtype: bool

In [319]:
df.sample(10)

,GAME_ID,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year
43758,20300741,22003,1610612765,DET,Detroit Pistons,2004-02-10,DET @ NJN,L,240,78,30,79,0.380,2,10.0,0.200,16,23,0.696,7.0,28.0,35.0,16,10.0,6.0,15.0,26.0,-11,89,28,69.0,0.405797,33.0,89.396226,86.264151,82,54.0,53,0.621540,32.941637,0.058363,0.058363,51.024669,2.975331,2.975331,2003-04
29867,29700289,21997,1610612742,DAL,Dallas Mavericks,1997-12-11,DAL vs. ORL,L,240,90,31,75,0.413,6,16.0,0.375,22,23,0.957,12.0,24.0,36.0,13,9.0,6.0,12.0,21.0,-10,100,25,59.0,0.423729,5.0,90.550000,98.700000,82,20.0,20,0.231688,4.633767,0.366233,0.366233,19.364679,0.635321,0.635321,1997-98
50278,20600197,22006,1610612756,PHX,Phoenix Suns,2006-11-26,PHX @ POR,W,240,119,46,85,0.541,11,25.0,0.440,16,19,0.842,9.0,32.0,41.0,28,10.0,3.0,12.0,20.0,18,101,35,60.0,0.583333,7.0,106.923077,104.461538,82,61.0,13,0.580292,7.543801,-0.543801,0.543801,47.040176,13.959824,13.959824,2006-07
7613,28700802,21987,1610612737,ATL,Atlanta Hawks,1988-04-01,ATL @ PHL,W,240,105,46,79,0.582,1,1.0,1.000,12,15,0.800,7.0,25.0,32.0,33,13.0,5.0,13.0,28.0,12,93,45,78.0,0.576923,44.0,108.128571,104.214286,82,50.0,70,0.625483,43.783801,0.216199,0.216199,51.505794,-1.505794,1.505794,1987-88
14133,29000385,21990,1610612743,DEN,Denver Nuggets,1991-01-02,DEN @ DET,L,240,107,40,90,0.444,6,15.0,0.400,21,25,0.840,8.0,28.0,36.0,23,9.0,0.0,15.0,24.0,-11,118,34,75.0,0.453333,6.0,125.233333,137.000000,82,20.0,30,0.222847,6.685422,-0.685422,0.685422,17.588065,2.411935,2.411935,1990-91
46064,20400964,22004,1610612764,WAS,Washington Wizards,2005-03-18,WAS @ MIL,L,240,90,34,86,0.395,6,22.0,0.273,16,23,0.696,17.0,26.0,43.0,16,9.0,3.0,11.0,20.0,-9,99,28,64.0,0.437500,35.0,100.587302,100.428571,82,45.0,63,0.505492,31.845979,3.154021,3.154021,44.604343,0.395657,0.395657,2004-05
5614,28500450,21985,1610612765,DET,Detroit Pistons,1986-01-17,DET vs. DEN,W,240,129,50,102,0.490,0,1.0,0.000,29,35,0.829,22.0,35.0,57.0,34,5.0,3.0,19.0,27.0,16,113,50,101.0,0.495050,18.0,115.230769,116.333333,82,46.0,39,0.466933,18.210376,-0.210376,0.210376,38.078106,7.921894,7.921894,1985-86
67798,21300842,22013,1610612765,DET,Detroit Pistons,2014-02-24,DET vs. GSW,L,239,96,40,99,0.404,4,13.0,0.308,12,18,0.667,16.0,29.0,45.0,26,7.0,6.0,8.0,23.0,-8,104,36,86.0,0.418605,23.0,100.649123,103.192982,82,29.0,57,0.414062,23.601519,-0.601519,0.601519,33.351543,-4.351543,4.351543,2013-14
40496,20201023,22002,1610612754,IND,Indiana Pacers,2003-03-26,IND vs. PHI,L,240,85,32,66,0.485,6,16.0,0.375,15,24,0.625,5.0,28.0,33.0,25,7.0,9.0,16.0,23.0,-6,91,26,50.0,0.520000,41.0,96.239437,93.549296,82,48.0,71,0.597331,42.410529,-1.410529,1.410529,47.570645,0.429355,0.429355,2002-03
43010,20300603,22003,1610612756,PHX,Phoenix Suns,2004-01-21,PHX @ DEN,L,240,92,42,95,0.442,2,11.0,0.182,6,9,0.667,18.0,25.0,43.0,24,13.0,6.0,12.0,23.0,-5,97,40,84.0,0.476190,16.0,93.162791,96.418605,82,29.0,43,0.382767,16.458989,-0.458989,0.458989,30.927921,-1.927921,1.927921,2003-04


In [320]:
df[AVG_STATS] = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[STATS].expanding().mean().reset_index((0,1))[STATS]
df[AVG_STATS] = df.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS].shift(1)
df = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).set_index('GAME_ID', drop=True)

In [321]:
df.head(10)[['PTS', 'AVG_PTS', 'AST', 'AVG_AST']]

,PTS,AVG_PTS,AST,AVG_AST
GAME_ID,,,,
28500005,91,NaN,25,NaN
28500013,91,91.000000,14,25.000000
28500030,102,91.000000,21,19.500000
28500045,105,94.666667,22,20.000000
28500054,114,97.250000,25,20.500000
28500060,113,100.600000,18,21.400000
28500069,114,102.666667,17,20.833333
28500076,119,104.285714,25,20.285714
28500085,97,106.125000,26,20.875000


In [327]:
df['perc_complete'] = (df['GP'].values-1) / df['TOTAL_GAMES']

In [328]:
home = df.loc[~df['MATCHUP'].str.contains('@')].sort_index()
away = df.loc[df['MATCHUP'].str.contains('@')].sort_index()

In [329]:
home['HOME'] = 1
away['HOME'] = 0

In [330]:
opp_headers = ['OPP_'+stat for stat in STATS+AVG_STATS]

In [331]:
away_opp = away.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})
home_opp = home.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})

In [332]:
data = pd.concat([pd.concat([home, away_opp[opp_headers]], axis=1), pd.concat([away, home_opp[opp_headers]], axis=1)])

In [333]:
data['W'] = data['WL'].map({'W': 1, 'L': 0})

In [334]:
with open('dat/locations.json') as f:
    locations = json.loads(f.read())
data['LOCATION'] = data['TEAM_NAME'].map(locations)
data['LOCATION_W_PCT'] = data['LOCATION'].map(data.loc[data['HOME'] == 1].groupby('LOCATION')['W'].mean().to_dict())

In [278]:
data = data.reset_index()

In [299]:
data = data.loc[data['perc_complete'] > 0]

In [335]:
data.loc[data[STATS+opp_headers+AVG_STATS+OPP_AVG_STATS].isna().any(axis=1)]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year,AVG_PTS,AVG_FG2M,AVG_FG2A,AVG_FG2_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,AVG_FT_PCT,AVG_DREB,AVG_OREB,AVG_AST,AVG_STL,AVG_BLK,AVG_TOV,AVG_PF,perc_complete,HOME,OPP_PTS,OPP_FG2M,OPP_FG2A,OPP_FG2_PCT,OPP_FG3M,OPP_FG3A,OPP_FG3_PCT,OPP_FTM,OPP_FTA,OPP_FT_PCT,OPP_DREB,OPP_OREB,OPP_AST,OPP_STL,OPP_BLK,OPP_TOV,OPP_PF,OPP_AVG_PTS,OPP_AVG_FG2M,OPP_AVG_FG2A,OPP_AVG_FG2_PCT,OPP_AVG_FG3M,OPP_AVG_FG3A,OPP_AVG_FG3_PCT,OPP_AVG_FTM,OPP_AVG_FTA,OPP_AVG_FT_PCT,OPP_AVG_DREB,OPP_AVG_OREB,OPP_AVG_AST,OPP_AVG_STL,OPP_AVG_BLK,OPP_AVG_TOV,OPP_AVG_PF,W,LOCATION,LOCATION_W_PCT
GAME_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20000001,22000,1610612752,NYK,New York Knicks,2000-10-31,NYK vs. PHI,L,238,72,25,70,0.357,3,11.0,0.273,19,24,0.792,14.0,23.0,37.0,14,6.0,4.0,22.0,30.0,-29,101,22,59.0,0.372881,0.0,72.0,101.0,82,48.0,1,0.008943,0.008943,-0.008943,0.008943,0.724383,47.275617,47.275617,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1,101,35,58.0,0.603448,3,8.0,0.375,22,30,0.733,29.0,8.0,27,10.0,5.0,13.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,New York,0.582055
20000002,22000,1610612751,NJN,New Jersey Nets,2000-10-31,NJN vs. CLE,L,239,82,31,85,0.365,3,10.0,0.300,17,26,0.654,12.0,35.0,47.0,24,9.0,8.0,12.0,31.0,-4,86,28,75.0,0.373333,0.0,82.0,86.0,82,26.0,1,0.340177,0.340177,-0.340177,0.340177,27.554326,-1.554326,1.554326,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1,86,30,71.0,0.422535,2,7.0,0.286,20,32,0.625,41.0,11.0,16,5.0,8.0,19.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,New Jersey,0.529301
20000003,22000,1610612753,ORL,Orlando Magic,2000-10-31,ORL vs. WAS,W,240,97,34,79,0.430,6,16.0,0.375,23,33,0.697,8.0,29.0,37.0,20,10.0,9.0,15.0,24.0,11,86,28,63.0,0.444444,1.0,97.0,86.0,82,43.0,1,0.842143,0.842143,0.157857,0.157857,69.213567,-26.213567,26.213567,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1,86,29,65.0,0.446154,4,7.0,0.571,16,24,0.667,35.0,9.0,20,6.0,1.0,26.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Orlando,0.583333
20000004,22000,1610612737,ATL,Atlanta Hawks,2000-10-31,ATL vs. CHH,L,241,82,30,81,0.370,6,15.0,0.400,16,21,0.762,11.0,18.0,29.0,14,9.0,2.0,13.0,31.0,-24,106,24,66.0,0.363636,0.0,82.0,106.0,82,25.0,1,0.027359,0.027359,-0.027359,0.027359,2.216071,22.783929,22.783929,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1,106,30,53.0,0.566038,5,9.0,0.556,31,40,0.775,37.0,7.0,16,7.0,6.0,17.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Atlanta,0.631135
20000005,22000,1610612761,TOR,Toronto Raptors,2000-10-31,TOR vs. DET,L,241,95,35,93,0.376,5,17.0,0.294,20,31,0.645,20.0,28.0,48.0,27,7.0,7.0,15.0,21.0,-9,104,30,76.0,0.394737,0.0,95.0,104.0,82,47.0,1,0.221137,0.221137,-0.221137,0.221137,17.912137,29.087863,29.087863,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1,104,41,81.0,0.506173,3,12.0,0.250,13,18,0.722,34.0,11.0,21,7.0,6.0,12.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Toronto,0.541387
20000006,22000,1610612741,CHI,Chicago Bulls,2000-10-31,CHI vs. SAC,L,240,81,26,67,0.388,6,8.0,0.750,23,29,0.793,7.0,22.0,29.0,19,8.0,4.0,19.0,20.0,-19,100,20,59.0,0.338983,0.0,81.0,100.0,82,15.0,1,0.050636,0.050636,-0.050636,0.050636,4.101516,10.898484,10.898484,2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1,100,35,60.0,0.583333,5,13.0,0.385,15,19,0.789,32.0,5.0,29,1

In [337]:
data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).head()[['GAME_DATE', 'TEAM_ABBREVIATION', 'MATCHUP', 'PTS', 'AVG_PTS', 'OPP_PTS', 'OPP_AVG_PTS']]

,GAME_DATE,TEAM_ABBREVIATION,MATCHUP,PTS,AVG_PTS,OPP_PTS,OPP_AVG_PTS
GAME_ID,,,,,,,
28500005,1985-10-25,ATL,ATL vs. WAS,91,NaN,100,NaN
28500013,1985-10-26,ATL,ATL @ MIL,91,91.000000,117,116.000000
28500030,1985-10-29,ATL,ATL vs. NYK,102,91.000000,87,89.000000
28500045,1985-11-01,ATL,ATL @ BOS,105,94.666667,109,110.333333
28500054,1985-11-02,ATL,ATL vs. PHL,114,97.250000,113,112.500000


*NOTE: OPP_AVG_PTS is not average points scored against, it is AVG_PTS for other team in matchup*

In [339]:
data.to_csv('dat/expanding_avg_1983-2017.csv')

In [144]:
df.loc[(df['SEASON_ID']==21984) & (df['TEAM_ABBREVIATION']=='GOS')].sort_values('GAME_DATE')

,index,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year,AVG_PTS,AVG_FG2M,AVG_FG2A,AVG_FG2_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,AVG_FT_PCT,AVG_DREB,AVG_OREB,AVG_AST,AVG_STL,AVG_BLK,AVG_TOV,AVG_PF,perc_complete
GAME_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28400006,2378,21984,1610612744,GOS,Golden State Warriors,1984-10-26,GOS vs. PHX,L,235,114,42,85,0.494,2,6.0,0.333,28,39,0.718,NaN,NaN,NaN,20,9.0,8.0,17.0,30.0,-8,122,40,79.0,0.506329,0.0,114.000000,122.000000,82,22.0,1,0.280212,0.280212,-0.280212,0.280212,22.697139,-0.697139,0.697139,1984-85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
28400008,2379,21984,1610612744,GOS,Golden State Warriors,1984-10-27,GOS @ DEN,L,240,121,48,94,0.511,2,3.0,0.667,23,27,0.852,15.0,30.0,45.0,20,9.0,6.0,29.0,30.0,-4,125,46,91.0,0.505495,0.0,117.500000,123.500000,82,22.0,2,0.333420,0.666840,-0.666840,0.666840,26.673599,-4.673599,4.673599,1984-85,114.000000,40.000000,79.000000,0.506329,2.000000,6.000000,0.333000,28.000000,39.000000,0.718000,NaN,NaN,20.000000,9.000000,8.000000,17.000000,30.000000,0.012195
28400020,2380,21984,1610612744,GOS,Golden State Warriors,1984-10-30,GOS @ LAL,L,240,110,49,111,0.441,0,3.0,0.000,12,20,0.600,23.0,19.0,42.0,22,12.0,1.0,18.0,24.0,-34,144,49,108.0,0.453704,0.0,115.000000,130.333333,82,22.0,3,0.149184,0.447551,-0.447551,0.447551,11.785497,10.214503,10.214503,1984-85,117.500000,43.000000,85.000000,0.505912,2.000000,4.500000,0.500000,25.500000,33.000000,0.785000,30.000000,15.000000,20.000000,9.000000,7.000000,23.000000,30.000000,0.024390
28400039,2381,21984,1610612744,GOS,Golden State Warriors,1984-11-01,GOS vs. SAN,L,240,108,46,96,0.479,0,1.0,0.000,16,25,0.640,23.0,28.0,51.0,26,10.0,2.0,24.0,26.0,-15,123,46,95.0,0.484211,0.0,113.250000,128.500000,82,22.0,4,0.147133,0.588532,-0.588532,0.588532,11.476379,10.523621,10.523621,1984-85,115.000000,45.000000,92.666667,0.488509,1.333333,4.000000,0.333333,21.000000,28.666667,0.723333,24.500000,19.000000,20.666667,10.000000,5.000000,21.333333,28.000000,0.036585
28400051,2382,21984,1610612744,GOS,Golden State Warriors,1984-11-03,GOS vs. UTH,W,240,112,46,100,0.460,1,2.0,0.500,19,29,0.655,16.0,32.0,48.0,24,5.0,3.0,6.0,25.0,5,107,45,98.0,0.459184,1.0,113.000000,124.200000,82,22.0,5,0.211723,1.058616,-0.058616,0.058616,17.302688,4.697312,4.697312,1984-85,113.250000,45.250000,93.250000,0.487434,1.000000,3.250000,0.250000,19.750000,27.750000,0.702500,25.666667,20.333333,22.000000,10.000000,4.250000,22.000000,27.500000,0.048780
28400075,2383,21984,1610612744,GOS,Golden State Warriors,1984-11-08,GOS vs. LAL,W,238,122,50,101,0.495,1,3.0,0.333,21,27,0.778,20.0,32.0,52.0,24,13.0,2.0,17.0,30.0,16,106,49,98.0,0.500000,2.0,114.500000,121.166667,82,22.0,6,0.312771,1.876625,0.123375,0.123375,25.770579,-3.770579,3.770579,1984-85,113.000000,45.200000,94.200000,0.481784,1.000000,3.000000,0.300000,19.600000,28.000000,0.693000,27.250000,19.250000,22.400000,9.000000,4.000000,18.800000,27.000000,0.060976
28400080,2384,21984,1610612744,GOS,Golden State Warriors,1984-11-09,GOS @ UTH,L,265,117,43,101,0.426,2,9.0,0.222,29,33,0.879,17.0,40.0,57.0,21,15.0,6.0,24.0,31.0,-8,125,41,92.0,0.445652,2.0,114.857143,121.714286,82,22.0,7,0.308615,2.160304,-0.160304,0.160304,25.146118,-3.146118,3.146118,1984-85,114.500000,45.833333,94.833333,0.484820,1.000000,3.000000,0.305500,19.833333,27.833333,0.707167,28.200000,19.400000,22.666667,9.666667,3.666667,18.500000,27.500000,0.073171
28400091,2385,21984,1610612744,GOS,Golden State Warriors,1984-11-11,GOS @ POR,L,240,97,36,80,0.450,0,5.0,0.000,25,29,0.862,8.0,27.0,35.0,18,10.0,3.0,24.0,41.0,-12,109,36,75.0,0.480000,2.0,112.625000,120.125000,82,22.0,8,0.289716,2.317724,-0.31

In [138]:
bad = []

In [139]:
for year in data['SEASON_ID'].unique():
    for team in data.loc[data['SEASON_ID']==year]['TEAM_ABBREVIATION'].unique():
        res = check_avg(data, year, team)
        if not np.all(res==(0, 0)):
            bad.append(year, team)

In [140]:
bad

[]